# Parsing the Pubmed Abstracts

In [2]:
import pubmed.downloader as pb
import json
from pprint import pprint
# spit abstracts to sentences
from nltk.tokenize import sent_tokenize

In [3]:
search_term = 'ACE inhibitor'
max_results = 3
query = pb.PubMedQuery(search_term, max_results)

In [4]:
??pb.PubMedQuery

In [9]:
ids = query.id_getter()
print ids

28605681,28602369,28602366


In [10]:
query.abstract_getter(ids)

{0: u'High-end analytical methods provide excellent data but may lack the robustness required in large analytical studies. In particular complex chemical matrices may cause difficulties and increase the need for extensive sample preparation. For screening of patients we thus developed a low-tech assay to monitor bradykinin degradation by serum proteases. The bradykinin concentration mirrors the activity of angiotensin-converting enzyme (ACE). Dabsylated bradykinin (DBK) and its labeled fragments DBK1-8 and DBK1-5 were visualized by thin-layer chromatography using only 3\u03bcL of serum. Lower DBK1-5 levels indicated reduced ACE activity due to medication (ACE-inhibitors) or disease. Provided that purified DBK is available, the assay protocol itself is very simple and does not require any expensive high-end equipment.',
 1: 'Factors predisposing the older person with acute myocardial infarction (MI) to develop heart failure (HF) include an increased prevalence of MI, multivessel coronar

In [13]:
%%time
#I just tried to download a few 
# if you don't want to download all abstracts, please do not use the download_all_abstracts() method, but rather the 
# abstract_getter() as below
pb.PubMedQuery.COUNT = 0
max_results = 10
query = pb.PubMedQuery(search_term, max_results)
ids = query.id_getter()
abstracts = query.abstract_getter(ids)

CPU times: user 152 ms, sys: 4 ms, total: 156 ms
Wall time: 8.98 s


In [17]:
abstracts

{0: u'Nonsteroidal antiinflammatory agents, \u03b2-lactam antibiotics, non-\u03b2 lactam antibiotics, and angiotensin-converting enzyme inhibitors are the most common classes of drugs that cause angioedema. Drug-induced angioedema is known to occur via mechanisms mediated by histamine, bradykinin, or leukotriene, and an understanding of these mechanisms is crucial in guiding therapeutic decisions. Nonallergic angioedema occurs in patients with genetic variants that affect metabolism or synthesis of bradykinin, substance P, prostaglandins, or leukotrienes, or when patients are taking drugs that have synergistic mechanisms. The mainstay in treatment of nonallergic drug-induced angioedema is cessation of the offending agents.',
 1: "Fabry's disease (FD) is a severe congenital metabolic disorder characterized by the deficient activity of lysosomal exoglycohydrolase alpha-galactosidase, characterized by glycosphingolipid deposition in several cells, such as capillary endothelial cells, rena

In [14]:
json_file = 'my_ten_abstracts.json'
print 'Saving to ' + json_file
with open(json_file, 'w') as outfile:
    json.dump(abstracts, outfile, indent=4)

Saving to my_ten_abstracts.json


## We might not need all the sentences in the abstract, so I am seeing what would happen if we just parse out the ones with key words, and write them to a textfile, and then parse the resulting file.

Keyword selection could happen here in the sentence tokenizer.

In [31]:
sentences = []
with open('my_ten_abstracts.json') as data_file:    
    data = json.load(data_file)
    for abstract in data.itervalues():
        sentences.append(sent_tokenize(abstract))
    # flatten the list of abstracts into one long list of sentences
    sentences = [sent for s in sentences for sent in s]
    print "Sentences: ",len(sentences)

Sentences:  91


In [95]:
%%writefile KeywordSentences.py
import json
keyword = 'ACE'
with open('my_ten_abstracts.json') as data_file:    
    data = json.load(data_file)
    #pick snippets related to ACE inhibitors
    for i in range(len(data.keys())):
        try:
            if keyword in data[str(i)]:
                print data[str(i)] + '\n'
            else:
                next
        except:
            next

Overwriting KeywordSentences.py


In [96]:
!python KeywordSentences.py > KeywordSentences.txt

In [97]:
#This mimics the format that the example parser file has
!cat KeywordSentences.txt

The renin-angiotensin-aldosterone system (RAAS) plays a key role in the pathophysiology of arterial hypertension as well as in more complex mechanisms of cardiovascular and renal diseases. RAAS-blocking agents like angiotensin-converting enzyme (ACE) inhibitors and angiotensin II receptor blockers, have long been key components in the treatment of essential hypertension, heart failure, diabetic nephropathy, and chronic kidney disease, showing benefits well beyond blood pressure reduction. Renin blockade as the first step of the RAAS cascade finally became possible in 2007 with the approval of aliskiren, the first orally active direct renin inhibitor available for clinical use and the newest antihypertensive agent on the market. In the last decade, many clinical trials and meta-analyses have been conducted concerning the efficacy and safety of aliskiren in comparison to other antihypertensive agents, as well as the efficacy and potential clinical use of various combinations. Large trial

In [98]:
! ./lexparser.sh KeywordSentences.txt

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.
Parsing file: KeywordSentences.txt
Parsing [sent. 1 len. 29]: The renin-angiotensin-aldosterone system -LRB- RAAS -RRB- plays a key role in the pathophysiology of arterial hypertension as well as in more complex mechanisms of cardiovascular and renal diseases .
(ROOT
  (S
    (NP
      (NP (DT The) (JJ renin-angiotensin-aldosterone) (NN system))
      (PRN (-LRB- -LRB-)
        (NP (NNP RAAS))
        (-RRB- -RRB-)))
    (VP (VBZ plays)
      (NP (DT a) (JJ key) (NN role))
      (PP
        (PP (IN in)
          (NP
            (NP (DT the) (NN pathophysiology))
            (PP (IN of)
              (NP (JJ arterial) (NN hypertension)))))
        (CONJP (RB as) (RB well) (IN as))
        (PP (IN in)
          (NP
            (NP
              (ADJP (RBR more) (JJ complex))
      

## Stanford Parser can also be run on straight JSON file...

In [93]:
# ! ./lexparser.sh pbabstract1.json

##  Next step: Sentiment Analysis!
<https://stanfordnlp.github.io/CoreNLP/>